In [1]:
#IMPORTAR DATASETS DE GOOGLE DRIVE

import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
dataTrain = pd.read_csv('/content/drive/MyDrive/Datasets/conjuntoEntrenamiento.csv', sep=';')
dataTest = pd.read_csv('/content/drive/MyDrive/Datasets/conjuntoTest.csv', sep=';')

dataTrain

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,titular,clickbait
0,Six children feared dead in Baltimore house fire,0
1,Day of the Mifids,1
2,The Greatest Late Night Snacks From Around the...,1
3,Can You Guess The Friends Season Based On Joey...,1
4,Dude trolls Starbucks baristas with a bunch of...,0
...,...,...
33774,Can You Match The Song Lyrics To The Disney TV...,1
33775,The Aspirational R.S.V.P.: Saying Yes When You...,1
33776,FreedomWorks Opposes Speaker Ryans Obamacare 2...,0
33777,More Californians dreaming of a country withou...,0


In [3]:
#COUNT VECTORIZER

from sklearn.feature_extraction.text import CountVectorizer 
 
countVectorizer = CountVectorizer() 

dfTrain = pd.DataFrame(dataTrain, index = None)
dfTest = pd.DataFrame(dataTest, index = None)
dfTrain.head()

train_x, train_y = dataTrain['titular'], dataTrain['clickbait']
test_x, test_y = dataTest['titular'], dataTest['clickbait']

train_x_vector = countVectorizer.fit_transform(train_x.values.astype('U'))
test_x_vector = countVectorizer.transform(test_x.values.astype('U'))

In [4]:
#TFIDF VECTORIZER

from sklearn.feature_extraction.text import TfidfVectorizer

train_x, train_y = dataTrain['titular'], dataTrain['clickbait']
test_x, test_y = dataTest['titular'], dataTest['clickbait']

tfidfVectorizer = TfidfVectorizer(stop_words='english')

train_x_vector = tfidfVectorizer.fit_transform(train_x.values.astype('U'))
test_x_vector = tfidfVectorizer.transform(test_x.values.astype('U'))

In [ ]:
#DICT VECTORIZER

from sklearn.feature_extraction import DictVectorizer

import numpy as np
import csv
from sklearn.metrics import roc_auc_score

def read_dataTrain(n, offset=0):
    X_dict, y = [], []
    with open('/content/drive/MyDrive/Datasets/conjuntoEntrenamiento.csv', 'r') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=';')
        for i in range(offset):
            next(reader)
        i = 0
        for row in reader:
            i += 1
            y.append(int(row['clickbait']))
            del row['clickbait']
            X_dict.append(row)
            if i >= n:
                break
    return X_dict, y

def read_dataTest(n, offset=0):
    X_dict, y = [], []
    with open('/content/drive/MyDrive/Datasets/conjuntoTest.csv', 'r') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=';')
        for i in range(offset):
            next(reader)
        i = 0
        for row in reader:
            i += 1
            y.append(int(row['clickbait']))
            del row['clickbait']
            X_dict.append(row)
            if i >= n:
                break
    return X_dict, y

nTrain = 21000
nTest = 7000
X_dict_train, y_train = read_dataTrain(nTrain)
X_dict_test, y_test = read_dataTest(nTest)

dtrain = dataTrain.iloc[:nTrain]
dtest = dataTrain.iloc[:nTest]

train_x, train_y = dtrain['titular'], dtrain['clickbait']
test_x, test_y = dtest['titular'], dtest['clickbait']

dictVectorizer = DictVectorizer(sparse=False)
train_x_vector = dictVectorizer.fit_transform(X_dict_train)
test_x_vector = dictVectorizer.transform(X_dict_test)

In [7]:
#SUPPORT VECTOR MACHINE

from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

svc = SVC(kernel='linear',probability=True)
svc.fit(train_x_vector, train_y)

score = svc.score(test_x_vector, test_y)
pred = svc.predict(test_x_vector)
pred_proba = svc.predict_proba(test_x_vector)
report = classification_report(test_y, pred, labels=[0,1])

print("Score: ", score)
print("F1 Score: ", f1_score(test_y, pred,labels=[0,1], average=None))
print("Report: ")
print(classification_report(test_y, pred, labels=[0,1]))

Score:  0.8364906464598627
F1 Score:  [0.79123205 0.86562226]
Report: 
              precision    recall  f1-score   support

           0       0.80      0.79      0.79      3327
           1       0.86      0.87      0.87      5119

    accuracy                           0.84      8446
   macro avg       0.83      0.83      0.83      8446
weighted avg       0.84      0.84      0.84      8446



In [5]:
#DECISION TREE

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

dec_tree = DecisionTreeClassifier()
dec_tree.fit(train_x_vector, train_y)

score = dec_tree.score(test_x_vector, test_y)
pred = dec_tree.predict(test_x_vector)
pred_proba = dec_tree.predict_proba(test_x_vector)
report = classification_report(test_y, pred, labels=[0,1])

print("Score: ", score)
print("F1 Score: ", f1_score(test_y, pred,labels=[0,1], average=None))
print("Report: ")
print(classification_report(test_y, pred, labels=[0,1]))

Score:  0.7925645275870234
F1 Score:  [0.72331017 0.83409091]
Report: 
              precision    recall  f1-score   support

           0       0.76      0.69      0.72      3327
           1       0.81      0.86      0.83      5119

    accuracy                           0.79      8446
   macro avg       0.79      0.77      0.78      8446
weighted avg       0.79      0.79      0.79      8446



In [ ]:
#NAIVE BAYES

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

gnb = GaussianNB()

#For Dict Vectorizer:

#gnb.fit(train_x_vector, train_y)
#score = gnb.score(test_x_vector, test_y)
#pred = gnb.predict(test_x_vector)
#pred_proba = gnb.predict_proba(test_x_vector)
#report = classification_report(test_y, pred, labels=[0,1])

#For Count Vectorizer or Tf-idf:

gnb.fit(train_x_vector.toarray(), train_y)
score = gnb.score(test_x_vector.toarray(), test_y)
pred = gnb.predict(test_x_vector.toarray())
pred_proba = gnb.predict_proba(test_x_vector.toarray())
report = classification_report(test_y, pred, labels=[0,1])

print("Score: ", score)
print("F1 Score: ", f1_score(test_y, pred,labels=[0,1], average=None))
print("Report: ")
print(classification_report(test_y, pred, labels=[0,1]))

In [6]:
#LOGISTIC REGRESSION

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

log_reg = LogisticRegression()
log_reg.fit(train_x_vector, train_y)

score = log_reg.score(test_x_vector, test_y)
pred = log_reg.predict(test_x_vector)
pred_proba = log_reg.predict_proba(test_x_vector)
report = classification_report(test_y, pred, labels=[0,1])

print("Score: ", score)
print("F1 Score: ", f1_score(test_y, pred,labels=[0,1], average=None))
print("Report: ")
print(classification_report(test_y, pred, labels=[0,1]))

Score:  0.8342410608572105
F1 Score:  [0.78580171 0.86481267]
Report: 
              precision    recall  f1-score   support

           0       0.80      0.77      0.79      3327
           1       0.86      0.87      0.86      5119

    accuracy                           0.83      8446
   macro avg       0.83      0.82      0.83      8446
weighted avg       0.83      0.83      0.83      8446



In [ ]:
#Libreria Anvil Uplink

!pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 8.0 MB/s 
     |████████████████████████████████| 51 kB 235 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=32d8f0d818c8cd343dc4a1cf93c6a4e651a9bb217f64bd6ad3928232dee6274c
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


In [ ]:
#Servicio web

import anvil.server

anvil.server.connect("LA6NSQZCSO5X27CWO6XZA6RH-YRUJISVSLLU7OHCF")

@anvil.server.callable
def check(headline):

  vector = countVectorizer.transform([headline])
  pred = log_reg.predict(vector)
  proba = log_reg.predict_proba(vector)
  
  #logs
  print("Headline: %s" % headline)
  print("Clickbait: %s" % pred[0])
  print("Proba: %s" % proba)
  print()

  return pred[0], proba
  

anvil.server.wait_forever()

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER
Headline: Here’s The Original Meaning Of 8 Phrases You Probably Use All The Time
Clickbait: 1
Proba: [[0.00375713 0.99624287]]

Headline: A 20-Year-Old Mother Pushing A Baby Stroller Was Shot In The Head And Killed In NYC
Clickbait: 0
Proba: [[0.710272 0.289728]]

Headline: I Have Monkeypox — Here’s What It’s Like And What You Should Know
Clickbait: 1
Proba: [[0.00463685 0.99536315]]

Headline: This Mom’s Heart Was Broken After Her Son’s Classmates Wouldn’t Sign His Yearbook, So People Are Coming Together To Brighten This 12-Year-Old’s Day
Clickbait: 1
Proba: [[0.00622118 0.99377882]]

Headline: This Mom’s Heart Was Broken After Her Son’s Classmates Wouldn’t Sign His Yearbook, So People Are Coming Together To Brighten This 12-Year-Old’s Day
Clickbait: 1
Proba: [[0.00622118 0.99377882]]

Headline: Court Decision Leaves Biden With Few Tools to Combat Climate Change
Clickbait: 0
Proba:

KeyboardInterrupt: ignored